In [ ]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

In [ ]:
from micropython import const
import struct
import bluetooth


_ADV_TYPE_FLAGS = const(0x01)
_ADV_TYPE_NAME = const(0x09)
_ADV_TYPE_UUID16_COMPLETE = const(0x3)
_ADV_TYPE_UUID32_COMPLETE = const(0x5)
_ADV_TYPE_UUID128_COMPLETE = const(0x7)
_ADV_TYPE_UUID16_MORE = const(0x2)
_ADV_TYPE_UUID32_MORE = const(0x4)
_ADV_TYPE_UUID128_MORE = const(0x6)
_ADV_TYPE_APPEARANCE = const(0x19)


def advertising_payload(limited_disc=False, br_edr=False, name=None, services=None, appearance=0):
    payload = bytearray()

    def _append(adv_type, value):
        nonlocal payload
        payload += struct.pack("BB", len(value) + 1, adv_type) + value

    _append(
        _ADV_TYPE_FLAGS,
        struct.pack("B", (0x01 if limited_disc else 0x02) + (0x18 if br_edr else 0x04)),
    )

    if name:
        _append(_ADV_TYPE_NAME, name)

    if services:
        for uuid in services:
            b = bytes(uuid)
            if len(b) == 2:
                _append(_ADV_TYPE_UUID16_COMPLETE, b)
            elif len(b) == 4:
                _append(_ADV_TYPE_UUID32_COMPLETE, b)
            elif len(b) == 16:
                _append(_ADV_TYPE_UUID128_COMPLETE, b)

    # See org.bluetooth.characteristic.gap.appearance.xml
    if appearance:
        _append(_ADV_TYPE_APPEARANCE, struct.pack("<h", appearance))

    return payload


def decode_field(payload, adv_type):
    i = 0
    result = []
    while i + 1 < len(payload):
        if payload[i + 1] == adv_type:
            result.append(payload[i + 2 : i + payload[i] + 1])
        i += 1 + payload[i]
    return result


def decode_name(payload):
    n = decode_field(payload, _ADV_TYPE_NAME)
    return str(n[0], "utf-8") if n else ""


def decode_services(payload):
    services = []
    for u in decode_field(payload, _ADV_TYPE_UUID16_COMPLETE):
        services.append(bluetooth.UUID(struct.unpack("<h", u)[0]))
    for u in decode_field(payload, _ADV_TYPE_UUID32_COMPLETE):
        services.append(bluetooth.UUID(struct.unpack("<d", u)[0]))
    for u in decode_field(payload, _ADV_TYPE_UUID128_COMPLETE):
        services.append(bluetooth.UUID(u))
    return services


In [7]:
import bluetooth
import random
import struct
import time
from machine import Pin, Timer
from micropython import const


_IRQ_CENTRAL_CONNECT = const(1)
_IRQ_CENTRAL_DISCONNECT = const(2)
_IRQ_GATTS_WRITE = const(3)

_FLAG_READ = const(0x0002)
_FLAG_WRITE_NO_RESPONSE = const(0x0004)
_FLAG_WRITE = const(0x0008)
_FLAG_NOTIFY = const(0x0010)

_UART_UUID = bluetooth.UUID("6E400001-B5A3-F393-E0A9-E50E24DCCA9E")
_UART_TX = (
    bluetooth.UUID("6E400003-B5A3-F393-E0A9-E50E24DCCA9E"),
    _FLAG_READ | _FLAG_NOTIFY,
)
_UART_RX = (
    bluetooth.UUID("6E400002-B5A3-F393-E0A9-E50E24DCCA9E"),
    _FLAG_WRITE | _FLAG_WRITE_NO_RESPONSE,
)
_UART_SERVICE = (
    _UART_UUID,
    (_UART_TX, _UART_RX),
)

is_ble_connected = False


class BLESimplePeripheral:
    def __init__(self, ble, name="mpy-uart"):
        self.led = Pin(2, Pin.OUT)
        self.timer1 = Timer(0)
        self.timer1.init(period=100, mode=Timer.PERIODIC, callback=lambda t: self.led.value(not self.led.value()))
        self._ble = ble
        self._ble.active(True)
        self._ble.irq(self._irq)
        ((self._handle_tx, self._handle_rx),) = self._ble.gatts_register_services((_UART_SERVICE,))
        self._connections = set()
        self._write_callback = None
        self._payload = advertising_payload(name=name, services=[_UART_UUID])
        self._advertise()

    def _irq(self, event, data):
        if event == _IRQ_CENTRAL_CONNECT:
            global is_ble_connected
            is_ble_connected = True
            self.led.value(1)
            self.timer1.deinit()
            conn_handle, _, _ = data
            print("New connection", conn_handle)
            self._connections.add(conn_handle)
        elif event == _IRQ_CENTRAL_DISCONNECT:
            global is_ble_connected
            is_ble_connected = False
            self.timer1.init(period=100, mode=Timer.PERIODIC, callback=lambda t: self.led.value(not self.led.value()))
            conn_handle, _, _ = data
            print("Disconnected", conn_handle)
            self._connections.remove(conn_handle)
            self._advertise()
        elif event == _IRQ_GATTS_WRITE:
            conn_handle, value_handle = data
            value = self._ble.gatts_read(value_handle)
            if value_handle == self._handle_rx and self._write_callback:
                self._write_callback(value)

    def send(self, data):
        for conn_handle in self._connections:
            self._ble.gatts_notify(conn_handle, self._handle_tx, data)

    def is_connected(self):
        return len(self._connections) > 0

    def _advertise(self, interval_us=500000):
        print("Starting advertising")
        self._ble.gap_advertise(interval_us, adv_data=self._payload)


if __name__ == "__main__":
    ble = bluetooth.BLE()
    p = BLESimplePeripheral(ble)

    i = 0
    while True:
        if p.is_connected():
            # Short burst of queued notifications.
            for _ in range(3):
                data = str(i) + "_"
                print("TX", data)
                p.send(data)
                i += 1
                time.sleep_ms(1000)

New connection 0
TX 24_
TX 25_
.TX 26_
TX 27_
Disconnected 0
Starting advertising
TX 28_
TX 29_
......New connection 0
TX 30_
TX 31_
TX 32_
TX 33_
TX 34_
TX 35_
TX 36_
TX 37_
.TX 38_
TX 39_
TX 40_
TX 41_
TX 42_
TX 43_
TX 44_
TX 45_
Disconnected 0
Starting advertising
.TX 46_
TX 47_
......................New connection 0
TX 48_
TX 49_
TX 50_
.TX 51_
TX 52_
TX 53_
TX 54_
TX 55_
TX 56_
TX 57_
TX 58_
.TX 59_
TX 60_
TX 61_
TX 62_
TX 63_
Disconnected 0
Starting advertising
TX 64_
TX 65_
.........................New connection 0
TX 66_
TX 67_
TX 68_
.TX 69_
TX 70_
TX 71_
TX 72_
TX 73_
TX 74_
TX 75_
TX 76_
.TX 77_
TX 78_
TX 79_
TX 80_
TX 81_
TX 82_
TX 83_
TX 84_
TX 85_
.TX 86_
TX 87_
TX 88_
TX 89_
TX 90_
TX 91_
TX 92_
TX 93_
TX 94_
.TX 95_
TX 96_
TX 97_
TX 98_
TX 99_
TX 100_
TX 101_
TX 102_
TX 103_
.TX 104_
TX 105_
TX 106_
TX 107_
TX 108_
TX 109_
TX 110_
TX 111_
TX 112_
TX 113_
.TX 114_
TX 115_
TX 116_
TX 117_
TX 118_
TX 119_
TX 120_
TX 121_
TX 122_
.TX 123_


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 97, in <module>
KeyboardInterrupt: 
